

<img src="img/logo.png" style="width:150px;height:150px"/>

# <center>IM458 - Tópicos em Métodos Numéricos: “Redes neurais informadas por Leis físicas como um novo paradigma para a solução de equações diferenciais parciais"</center></h1>

## Gabriel Welfany Rodrigues - RA: 193275
## José Eduardo Chiarelli Bueno Filho - RA: 192727

## Notebook Setup

In [2]:
import sys
import time
import scipy.io
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from pyDOE import lhs
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Fluxo Bifásico em Meio Poroso

## 1. Indrodução
Simulações numéricas de reservatórios tem evoluido por mais de 40 anos. Estes processos são utilizadas para estimar características de produção, calibrar parametros dos reservatórios, vizualizar padrões de fluxo dos reservatórios etc. Possibilitando, desta forma, a obtenção de informações para maximizar o processo de extração de oléo e gás. 

Devido a natureza física e a complexidade dos modelos matemáticos envolvidos, simulações de reservatórios podem se tornar custosas, tanto em termos de tempo quanto em memória. Métodos que buscam a redução destes custos são o foco de muitos estudos e novas soluções têm se mostrado promissoras, como, por exemplo, o uso de redes neurais para a solução das equações diferenciais parciais envolvidas no modelo. Estas redes são as chamadas *Physics Informed Neural Networks* (PINN's), onde as equações diferenciais parciais provenientes do modelo físico são utilizadas para auxiliar o treinamento da rede.
     
Portanto, o objetivo do presente trabalho é aplicar uma PINN para resolver um problema de escoamento bifásico em meio poroso. A princípio é considerado um escoamento unidimensional, incompressível e imiscível em um meio homogêneo e isotrópico. A razão para se adotar tal modelo é o de trabalhar com equações simplificadas, visando facilitar a compreensão dos procedimentos e princípios da aplicação de uma PINN ao problema em questão. 

## 2. Definição do problema 
Nesta seção, a formulação do problema que se pretende resolver com a PINN é apresentada. Basicamente, o que se deseja obter são os comportamentos (no espaço e no tempo) da pressão e da saturação de água no interior de um reservatório unidimensional preenchido de óleo sob a ação de um poço injetor e de um poço produtor localizados em suas extremidades. O poço injetor injeta água à uma taxa constante e o poço produtor extrai uma combinação de óleo e água também a uma taxa constante. Dessa forma, desprezando efeitos gravitacionais, as equações diferenciais parciais que governam este sistema são dadas à seguir

\begin{equation}
\tag{1}
\left\{\begin{matrix}
\hspace{-0.5cm}-\nabla\cdot K\lambda(s)\nabla p=q, \\ 
\phi\frac{\partial s}{\partial t}+\nabla\cdot (f(s)v)=\frac{q_{w}}{\rho_{w}} \\
v = -K\lambda(s)\nabla p
\end{matrix}\right.
\end{equation}

onde $K$ é a permeabilidade do meio, $\lambda(s)$ é a mobilidade total do fluido no meio, $p$ é a pressão, $q$ é a taxa de injeção de água, $\phi$ é a porosidade do meio, $s$ é a saturação de água, $v$ é a velocidade total do fluído no meio e $q_{w}/\rho_{w}$ é a taxa de produção de água. 

A mobilidade total é a soma das mobilidades da água e do óleo no meio, sendo expressa por:

\begin{equation}
\tag{2}
\lambda(s) = \lambda_{w}(s) + \lambda_{o}(s), \quad \lambda_{w}(s) = \frac{s^{2}}{\mu_w}, \quad \lambda_{o}(s) = \frac{(1-s)^{2}}{\mu_o}
\end{equation}

onde $\mu_w$ e $\mu_o$ são as viscosidades da água e do óleo, respectivamente.

A taxa de produção de água pode ser expressa com uma função de $q$:

\begin{equation}
\tag{3}
\frac{q_w}{\rho_w}=max(q,0)+f(s)min(q,0).
\end{equation}

onde $f(s)$ é chamada de função de fluxo fracionário, cuja expressão é dada por

\begin{equation}
\tag{4}
f(s) = \frac{\lambda_{w}(s)}{\lambda_{w}(s) + \lambda_{o}(s)}
\end{equation}

Como o problema considerado é unidimensional, homogêneo e isotrópico os parâmetros do meio poroso são constantes; adotou-se $K = 1$ e $\phi = 1/0.375$, fazendo com que as equações do sistema (1) possam ser simplificadas da seguinte maneira:

\begin{equation}
\tag{5}
\left\{\begin{matrix}
\hspace{-0.5cm}\frac{\partial}{\partial x}\left( \lambda(s)\frac{\partial p}{\partial x}  \right) = q\\ 
\frac{1}{0.375}\frac{\partial s}{\partial t} + \frac{\partial}{\partial x}\left ( f(s)v \right ) = max(q,0)+f(s)min(q,0)\\
v = -\lambda(s)\frac{\partial p}{\partial x}
\end{matrix}\right.
\end{equation}

### 2.1. Condições iniciais e de contorno
Foi considerado que o reservatório possui uma certa quantidade de água antes dos poços serem colocados em funcionamento (água conata), e que a saturação desta água decai linearmente ao longo do reservatório. Assim, a condição inicial para a saturação de água possui a seguinte forma

\begin{equation}
\tag{6}
s(x, 0) = 0.1 - 0.1x
\end{equation}

No presente trabalho, realiza-se a simulação até o tempo no qual a água injetada chega ao poço produtor, permitindo, desse modo, que se assuma a seguinte condição de contorno para a saturação de água:

\begin{equation}
\tag{7}
s(1, t) = 0
\end{equation}

Para a determinação das condições de contorno da pressão assumiu-se que 

\begin{equation}
\tag{8}
p(0, t) = 0
\end{equation}

pois as pressões que se trabalham na simulação são pressões relativas e não pressões totais, logo, a utilização da condição acima não acarreta em perda de generalidade. Para a condição do contorno superior ($x = 1$), utilizou-se o a taxa de produção, ou seja

\begin{equation}
\tag{9}
-\left[ \frac{\partial}{\partial x}\left( \lambda(s)\frac{\partial p}{\partial x}  \right) \right]_{x=1}  = -1
\end{equation}

Já para a condição inicial, assumiu-se que o termo fonte $q(x, t)$ se comporta como uma função pontual e constante no tempo, ou seja

\begin{equation}
q(x, t) = q(x) = \delta(x) - \delta(x-1)
\end{equation}

Agora, como os valores absolutos das taxas de injeção são unitários, o valor do fluxo $v$ no interior do reservatório também deve ser unitário e observando-se que $q = \frac{\partial{v}}{\partial{x}}$ pode se concluir que

\begin{equation}
v(x) = \int q(x)dx \Rightarrow v(x) = \theta(x) - \theta(x-1)
\end{equation}

onde $\theta(x)$ é a função degrau. Finalmente, observando-se que $\frac{\partial{p}}{\partial{x}} = -\frac{v}{\lambda(s)}$ e que $s(x, t=0) \approx 0 \Rightarrow \lambda(0) = \frac{1}{\mu_{o}}$ a pressão inicial pode ser dada por


\begin{equation}
\tag{10}
p(x, 0) = -\mu_{o}\int v(x)dx \Rightarrow p(x, 0) = -\mu_{o}x, \quad x \in [0, 1]
\end{equation}

Assim, as equações (6-10) formam as condições iniciais e de contorno do problema.

### 2.2. Equações dos resíduos
Partindo da definição de $q(x)$ dada na seção anterior e notando que

\begin{equation}
\frac{\partial}{\partial x}(f(s)v) = v\frac{\partial f(s)}{\partial x} + f(s)\frac{\partial v}{\partial x} = \frac{\partial f(s)}{\partial x}
\end{equation}

pois $v = 1$ e $\frac{\partial v}{\partial x} = q = 0$ para $x \in ]0, 1[$, as equações de resíduo $f_{p}(x, t)$ e $f_s(x, t)$ para a pressão e a saturação, respectivamente, são dadas por

\begin{equation}
\tag{11}
\left\{\begin{matrix}
\hspace{-0.5cm}f_{p}(x, t) = \frac{\partial}{\partial x}\left( \lambda(s)\frac{\partial p}{\partial x}  \right) = 0\\ 
f_s(x, t) = \frac{1}{0.375}\frac{\partial s}{\partial t} + \frac{\partial f(s)}{\partial x} = 0\\
\end{matrix}\right.
\end{equation}

## 3. Resultados
### 3.1 Caso simplificado (proposto)
#### 3.1.1 Discussão
Nesta seção mostraremos os resultados alcançados utilizando a estrutura de redes neurais adaptada para o problema simplificado (proposto), ou seja, com os parâmetros físicos do meio poroso constantes. Os valores de viscosidade de água e óleo utilizadas foram $\mu_{w} = 0.09$ e $\mu_{o} = 0.9$, respectivamente.

A Figura 1 illustra os resultados alcançados para a pressão enquanto a Figura 2 illustra os resultados para a saturação. Para esta solução, foram utilizados 100 pontos para as condições iniciais (de 256 disponíveis), 50 pontos para as condições de contorno (de 128 disponíveis) e 10000 pontos de colocação (de 32768 disponíveis), todos radomicamente distribuídos. Com relação a arquitetura da rede, foram utilizadas 4 *hidden layers* com 100 neurônios, cada um utilizando tangente hiperbólica como função de ativação. Quanto ao treinamento, o otimizador Adams foi utilizado para uma aproximação inicial (1000 iterações) seguido pelo otimizador L-BFGS-B.   

<img src="img/pressure_best-v2.jpeg" style="width:700px;height:500px"/> 
<center>Figura 1 - Superior: Solução prevista pela rede neural para pressão. Inferior: Comparação entre a solução exata e a prevista para três tempos.</center>

<img src="img/saturation_best-v2.jpeg" style="width:700px;height:500px"/>
<center>Figura 2 - Superior: Solução prevista pela rede neural para saturação. Inferior: Comparação entre a solução exata e a prevista para três tempos.</center>


Também foram realizados testes alterando alguns hiperparâmetros da rede, tais como largura, profundidade e número de pontos de colocação. Os resultados para os testes mencionados são apresentados nas Tabelas 1 à 3.

##### <center>Tabela 1 - Erro relativo $\mathscr{L}_{2}$ entre resultado previsto e a solução exata para a pressão e saturação considerando 4 hidden layers com diferentes quantidades de neurônios. </center> 

| Neurônios | Erro - Pressão    | Erro - Saturação  |
|-----------|-------------------|-------------------|
| 100       |$2.58\times10^{-1}$|$6.99\times10^{-2}$|
| 200       |$2.85\times10^{-1}$|$9.67\times10^{-2}$|
| 300       |$1.77\times10^{-1}$|$1.42\times10^{-1}$|

##### <center>Tabela 2 - Erro relativo $\mathscr{L}_{2}$ entre resultado previsto e a solução exata para a pressão e saturação considerando diferentes quantidades de hidden layers com 100 neurônios. </center> 

| Layers    | Erro - Pressão    | Erro - Saturação  |
|-----------|-------------------|-------------------|
| 2         |$2.58\times10^{-1}$|$1.46\times10^{-1}$|
| 4         |$2.58\times10^{-1}$|$6.99\times10^{-2}$|
| 8         |$3.90\times10^{-1}$|$1.05\times10^{-1}$|
| 10        |$4.67\times10^{-1}$|$1.02\times10^{-1}$|

##### <center>Tabela 3 - Erro relativo $\mathscr{L}_{2}$ entre resultado previsto e a solução exata para a pressão e saturação considerando diferentes quantidades de pontos de colocação para 4 hidden layers com 100 neurônios cada. </center> 

| Nf        | Erro - Pressão    | Erro - Saturação  |
|-----------|-------------------|-------------------|
| 10000     |$2.58\times10^{-1}$|$6.99\times10^{-2}$|
| 15000     |$1.87\times10^{-1}$|$6.56\times10^{-2}$|
| 20000     |$1.55\times10^{-1}$|$5.98\times10^{-2}$|
| 30000     |$9.98\times10^{-2}$|$4.35\times10^{-2}$|

#### 3.1.2 Implementação

In [ ]:
#!/usr/bin/env python3

import time
import scipy.io
import numpy as np
import tensorflow as tf
from pyDOE import lhs
from scipy.interpolate import griddata

np.random.seed(1234)
tf.set_random_seed(1234)

class PhysicsInformedNN:
    def __init__(self, x0, p0, s0, p_ub, s_lb, tb, X_f, layers, lb, ub, mu_o, mu_w):
        X0 = np.concatenate((x0, 0*x0), 1)  # (x0, 0)
        X_lb = np.concatenate((0*tb + lb[0], tb), 1)  # (lb[0], tb)
        X_ub = np.concatenate((0*tb + ub[0], tb), 1)  # (ub[0], tb)
        
        self.x0 = X0[:, 0:1]
        self.t0 = X0[:, 1:2]
        self.x_lb = X_lb[:, 0:1]
        self.t_lb = X_lb[:, 1:2]
        self.x_ub = X_ub[:, 0:1]
        self.t_ub = X_ub[: ,1:2]
        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]
        self.p0 = p0
        self.s0 = s0
        self.p_ub = p_ub
        self.s_lb = s_lb
        self.layers = layers
        self.lb = lb
        self.ub = ub
        self.mu_o = mu_o
        self.mu_w = mu_w

        self.weights, self.biases = self.initialize_NN(layers)

        self.x0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.x0.shape[1]])
        self.t0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.t0.shape[1]])
        self.x_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.x_lb.shape[1]])
        self.t_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.t_lb.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.x_ub.shape[1]])
        self.t_ub_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.t_ub.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.t_f.shape[1]])
        self.p0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.p0.shape[1]])
        self.s0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.s0.shape[1]])
        self.p_ub_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.p_ub.shape[1]])
        self.s_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.s_lb.shape[1]])
        
        self.p0_pred, self.s0_pred = self.net_ps(self.x0_tf, self.t0_tf)
        self.p_lb_pred, self.s_lb_pred = self.net_ps(self.x_lb_tf, self.t_lb_tf)
        self.p_ub_pred, _ = self.net_f_ps(self.x_ub_tf, self.t_ub_tf)
        _, self.s_ub_pred = self.net_ps(self.x_ub_tf, self.t_ub_tf)
        self.f_p_pred, self.f_s_pred = self.net_f_ps(self.x_f_tf, self.t_f_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.p0_tf - self.p0_pred)) + \
                    tf.reduce_mean(tf.square(self.s0_tf - self.s0_pred)) + \
                    tf.reduce_mean(tf.square(self.p_lb_pred)) + \
                    tf.reduce_mean(tf.square(-1 - self.p_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.s_lb_tf - self.s_lb_pred)) + \
                    tf.reduce_mean(tf.square(self.s_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.f_p_pred)) + \
                    tf.reduce_mean(tf.square(self.f_s_pred))            

        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                                method = 'L-BFGS-B',
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0*np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
    
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))

        init = tf.global_variables_initializer()
        self.sess.run(init)


    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
            
        return weights, biases


    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)


    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
            
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        
        return Y


    def net_ps(self, x, t):
        ps = self.neural_net(tf.concat([x, t], 1), self.weights, self.biases)
        p = ps[:, 0:1]
        s = ps[:, 1:2]
        
        return p, s
    
    
    def net_f_ps(self, x, t):
        p, s = self.net_ps(x, t)
        p_x = tf.gradients(p, x)[0]
        lp_xx = tf.gradients(self.lbd(s)*p_x, x)[0]
        f_x = tf.gradients(self.ff(s), x)[0]
        s_t = tf.gradients(s, t)[0]
        
        fp = -lp_xx
        fs = (1/0.375)*s_t + f_x

        return fp, fs
    
    
    def lbd(self, s):  # mobilidade (lambda)
        lbd = s**2/self.mu_w + (1-s)**2/self.mu_o
        
        return lbd
    
    
    def ff(self, s):  # fractional flow -> f(s)
        ff = (s**2/self.mu_w)/self.lbd(s)
        
        return ff
    

    def callback(self, loss):
        print('Loss:', loss)


    def train(self, nIter):
        tf_dict = {self.x0_tf: self.x0,
                   self.t0_tf: self.t0,
                   self.x_lb_tf: self.x_lb,
                   self.t_lb_tf: self.t_lb,
                   self.x_ub_tf: self.x_ub,
                   self.t_ub_tf: self.t_lb,
                   self.x_f_tf: self.x_f,
                   self.t_f_tf: self.t_f,
                   self.p0_tf: self.p0,
                   self.s0_tf: self.s0,
                   self.p_ub_tf: self.p_ub,
                   self.s_lb_tf: self.s_lb}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)

            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                print('It: %d, Loss: %.3e, Time: %.2f' %
                      (it, loss_value, elapsed))
                start_time = time.time()

        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)


    def predict(self, X_star):
        p_star = self.sess.run(self.p0_pred, {self.x0_tf: X_star[:,0:1], 
                                              self.t0_tf: X_star[:,1:2]})
        f_p_star = self.sess.run(self.f_p_pred, {self.x_f_tf: X_star[:,0:1], 
                                                 self.t_f_tf: X_star[:,1:2]})
        s_star = self.sess.run(self.s0_pred, {self.x0_tf: X_star[:,0:1], 
                                              self.t0_tf: X_star[:,1:2]})
        f_s_star = self.sess.run(self.f_s_pred, {self.x_f_tf: X_star[:,0:1], 
                                                 self.t_f_tf: X_star[:,1:2]})

        return p_star, f_p_star, s_star, f_s_star
    
    
if __name__ == "__main__":
    mu_w = 0.09
    mu_o = 0.9

    N0 = 100
    N_b = 50
    N_f = 10000
    layers =  [2, 100, 100, 2]

    data = scipy.io.loadmat('2phaseflow.mat')

    t = data['tt'].flatten()[:, None]
    x = data['x'].flatten()[:, None]
    Exact_p = np.real(data['P_history'])
    Exact_s = np.real(data['S_history'])

    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    p_star = Exact_p.T.flatten()[:, None]
    s_star = Exact_s.T.flatten()[:, None]
    
    idx_x = np.random.choice(x.shape[0], N0, replace=False)
    x0 = x[idx_x, :]
    p0 = -mu_o*x0
    s0 = 0.1 - 0.1*x0
        
    idx_t = np.random.choice(t.shape[0], N_b, replace=False)
    tb = t[idx_t, :]
    
    p_lb = 0*tb
    p_ub = Exact_p[-1:, idx_t]
    s_lb = Exact_s[0:1, idx_t]

    lb = X_star.min(0)
    ub = X_star.max(0)
    lbp = lb + np.array((1/x.size, 0))
    ubp = ub - np.array((1/x.size, 0))
    X_f = lbp + (ubp-lbp)*lhs(2, N_f)

    model = PhysicsInformedNN(x0, p0, s0, p_ub, s_lb, tb, X_f, layers, lb, ub, mu_o, mu_w)

    start_time = time.time()
    model.train(1000)
    elapsed = time.time() - start_time
    print('Training time: %.4f' % (elapsed))

    p_pred, f_p_pred, s_pred, f_s_pred = model.predict(X_star)

    error_p = np.linalg.norm(p_star - p_pred, 2)/np.linalg.norm(p_star, 2)
    print('Error p: %e' % (error_p))
    
    error_s = np.linalg.norm(s_star - s_pred, 2)/np.linalg.norm(s_star, 2)
    print('Error s: %e' % (error_s))

    P_pred = griddata(X_star, p_pred.flatten(), (X, T), method='cubic')
    S_pred = griddata(X_star, s_pred.flatten(), (X, T), method='cubic')

    np.savez('2phaseflow.npz', x0=x0, tb=tb, lb=lb, ub=ub, P_pred=P_pred,
             S_pred=S_pred, Exact_p=Exact_p, Exact_s=Exact_s, t=t, x=x)

In [ ]:
data = np.load('2phaseflow.npz')
x0 = data['x0']
tb = data['tb']
lb = data['lb']
ub = data['ub']
P_pred = data['P_pred']
S_pred = data['S_pred']
Exact_p = data['Exact_p']
Exact_s = data['Exact_s']
t = data['t']
x = data['x']

X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
X_u_train = np.vstack([X0, X_lb, X_ub])

########################################
####### Pressure #######################
########################################

fig, ax = plt.subplots()
ax.axis('off')

####### Row 0: p(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])

h = ax.imshow(P_pred.T, interpolation='nearest', cmap='viridis',
              extent=[lb[1], ub[1], lb[0], ub[0]],
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:, 1], X_u_train[:, 0], 'kx',
        label = 'Data (%d points)' % (X_u_train.shape[0]),
        markersize = 4, clip_on = False)

line = np.linspace(x.min(), x.max(), 2)[:, None]
ax.plot(t[15]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[60]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[120]*np.ones((2, 1)), line, 'k--', linewidth = 1)

ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
ax.set_title('$p(t,x)$', fontsize = 10)

####### Row 1: p(t,x) slices ##################    
gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.9, wspace=1.0)

ax = plt.subplot(gs1[0, 0])
ax.plot(x, Exact_p[:, 15], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, P_pred[15, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$p(t,x)$')
ax.set_title('$t = %.2f$' % (t[15]), fontsize = 10)
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-1.1, 0.1])

ax = plt.subplot(gs1[0, 1])
ax.plot(x, Exact_p[:, 60], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, P_pred[60, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$p(t,x)$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-1.1, 0.1])
ax.set_title('$t = %.2f$' % (t[60]), fontsize = 10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.8),
          ncol=5, frameon=False)

ax = plt.subplot(gs1[0, 2])
ax.plot(x, Exact_p[:, 120], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, P_pred[120, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$p(t,x)$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-1.1, 0.1])
ax.set_title('$t = %.2f$' % (t[120]), fontsize = 10)

########################################
####### Saturation #####################
########################################

fig, ax = plt.subplots()
ax.axis('off')

####### Row 0: s(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])

h = ax.imshow(S_pred.T, interpolation='nearest', cmap='viridis',
              extent=[lb[1], ub[1], lb[0], ub[0]],
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:, 1], X_u_train[:, 0], 'kx',
        label = 'Data (%d points)' % (X_u_train.shape[0]),
        markersize = 4, clip_on = False)

line = np.linspace(x.min(), x.max(), 2)[:, None]
ax.plot(t[15]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[60]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[120]*np.ones((2, 1)), line, 'k--', linewidth = 1)

ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
ax.set_title('$S(t,x)$', fontsize = 10)

####### Row 1: s(t,x) slices ##################    
gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.9, wspace=0.5)

ax = plt.subplot(gs1[0, 0])
ax.plot(x, Exact_s[:, 15], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, S_pred[15, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$S(t,x)$')
ax.set_title('$t = %.2f$' % (t[15]), fontsize = 10)
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])

ax = plt.subplot(gs1[0, 1])
ax.plot(x, Exact_s[:, 60], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, S_pred[60, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$S(t,x)$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])
ax.set_title('$t = %.2f$' % (t[60]), fontsize = 10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.8),
          ncol=5, frameon=False)

ax = plt.subplot(gs1[0, 2])
ax.plot(x, Exact_s[:, 120], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, S_pred[120, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$S(t,x)|$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])
ax.set_title('$t = %.2f$' % (t[120]), fontsize = 10) 

### 3.2 Caso real
#### 3.2.1 Discussão
Nesta seção serão apresentados resultados referentes à aplicação da PINN à um problema onde as propriedades físicas do meio poroso não são constantes, ou seja, um problema onde estas propriedades variam com a posição dentro do reservatório. Os valores para a porosidade do meio foram tomadas à partir de um banco de dados chamado SPE10, utilizado como *benchmark* para simuladores de reservatórios e cujos dados são obtidos a partir de medições em reservatórios reais. Os valores utilizados para a porosidade são mostrados na Figura 3.

<img src="img/phi.png" style="width:500px;height:300px"/> 
<center>Figura 3 - Valores de porosidade.</center>

Quanto a permeabilidade, a simulação teve de ser realizada com valores sintéticos (1 ciclo de senóide), pois não houve convergência da PINN com valores reais obtidos a partir do banco de dados. Os valores utilizados para a permeabilidade são mostrados na Figura 4.

<img src="img/K.png" style="width:500px;height:300px"/> 
<center>Figura 4 - Valores de permeabilidade.</center>

De forma a se obter valores de pressão menores ou iguais a 1 na saída, os valores de viscosidade de água e óleo utilizadas foram  $\mu_{w}=0.0855$  e  $\mu_{o}=0.855$.

Para a realização da simulação foi utilizada uma rede com 4 camadas com 100 neurônios, dado que esta configuração gerou bons resultados para o caso simplificado. Foram utilizados, também, 100 pontos para as condições iniciais (de 220 disponíveis), 50 pontos para as condições de contorno (de 128 disponíveis) e 10000 pontos de colocação (de 28160 disponíveis). Os resultados obtidos são mostrados nas Figuras 5 e 6 e os valores obtidos para os erros são mostrados na Tabela 4.

<img src="img/pressure_spe10_sineK.jpeg" style="width:700px;height:500px"/> 
<center>Figura 5 - Superior: Solução prevista pela rede neural para pressão. Inferior: Comparação entre a solução exata e a prevista para três tempos.</center>

<img src="img/saturation_spe10_sineK.jpeg" style="width:700px;height:500px"/>
<center>Figura 6 - Superior: Solução prevista pela rede neural para saturação. Inferior: Comparação entre a solução exata e a prevista para três tempos.</center>

| Erro - Pressão    | Erro - Saturação  |
|-------------------|-------------------|
|$2.45\times10^{-1}$|$2.77\times10^{-1}$|

#### 3.2.2 Implementação

In [ ]:
#!/usr/bin/env python3

import time
import scipy.io
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from pyDOE import lhs
from scipy.interpolate import griddata

np.random.seed(1234)
tf.set_random_seed(1234)

class PhysicsInformedNN:
    def __init__(self, x0, p0, s0, p_lb, p_ub, s_lb, tb, 
                 X_f, layers, lb, ub, mu_o, mu_w, K, phi):
        X0 = np.concatenate((x0, 0*x0), 1)  # (x0, 0)
        X_lb = np.concatenate((0*tb + lb[0], tb), 1)  # (lb[0], tb)
        X_ub = np.concatenate((0*tb + ub[0], tb), 1)  # (ub[0], tb)
        
        self.x0 = X0[:, 0:1]
        self.t0 = X0[:, 1:2]
        self.x_lb = X_lb[:, 0:1]
        self.t_lb = X_lb[:, 1:2]
        self.x_ub = X_ub[:, 0:1]
        self.t_ub = X_ub[: ,1:2]
        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]
        self.p0 = p0
        self.s0 = s0
        self.p_lb = p_lb
        self.p_ub = p_ub
        self.s_lb = s_lb
        self.layers = layers
        self.lb = lb
        self.ub = ub
        self.mu_o = mu_o
        self.mu_w = mu_w
        self.K = K
        self.phi = phi

        self.weights, self.biases = self.initialize_NN(layers)

        self.x0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.x0.shape[1]])
        self.t0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.t0.shape[1]])
        self.x_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.x_lb.shape[1]])
        self.t_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.t_lb.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.x_ub.shape[1]])
        self.t_ub_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.t_ub.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.t_f.shape[1]])
        self.p0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.p0.shape[1]])
        self.s0_tf = tf.placeholder(tf.float32,
                                    shape=[None, self.s0.shape[1]])
        self.p_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.p_lb.shape[1]])
        self.p_ub_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.p_ub.shape[1]])
        self.s_lb_tf = tf.placeholder(tf.float32,
                                      shape=[None, self.s_lb.shape[1]])
        self.K_tf = tf.placeholder(tf.float32, shape=[None])
        self.phi_tf = tf.placeholder(tf.float32, shape=[None])
        
        self.p0_pred, self.s0_pred = self.net_ps(self.x0_tf, self.t0_tf)
        self.p_lb_pred, self.s_lb_pred = self.net_ps(self.x_lb_tf, self.t_lb_tf)
        self.p_ub_pred, _ = self.net_f_ps(self.x_ub_tf, self.t_ub_tf)
        _, self.s_ub_pred = self.net_ps(self.x_ub_tf, self.t_ub_tf)
        self.f_p_pred, self.f_s_pred = self.net_f_ps(self.x_f_tf, self.t_f_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.p0_tf - self.p0_pred)) + \
                    tf.reduce_mean(tf.square(self.s0_tf - self.s0_pred)) + \
                    tf.reduce_mean(tf.square(self.p_lb_pred)) + \
                    tf.reduce_mean(tf.square(-1 - self.p_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.s_lb_tf - self.s_lb_pred)) + \
                    tf.reduce_mean(tf.square(self.s_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.f_p_pred)) + \
                    tf.reduce_mean(tf.square(self.f_s_pred)) 

        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                                method = 'L-BFGS-B',
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0*np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))

        init = tf.global_variables_initializer()
        self.sess.run(init)


    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
            
        return weights, biases


    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)


    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
            
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        
        return Y


    def net_ps(self, x, t):
        ps = self.neural_net(tf.concat([x, t], 1), self.weights, self.biases)
        p = ps[:, 0:1]
        s = ps[:, 1:2]
        
        return p, s
        
    
    def net_s_lb(self, x, t):  # lower boundary condition on s
        p, s = self.net_ps(x, t)
        s_t = tf.gradients(s, t)[0]
        
        return self.por(x)*s_t + self.ff(s)
    
    
    def net_f_ps(self, x, t):
        p, s = self.net_ps(x, t)
        p_x = tf.gradients(p, x)[0]
        p_xx = tf.gradients(self.perm(x)*self.lbd(s)*p_x, x)[0]
        f_x = tf.gradients(self.ff(s), x)[0]
        s_t = tf.gradients(s, t)[0]
        
        fp = -p_xx
        fs = self.por(x)*s_t + f_x

        return fp, fs
    
    
    def perm(self, x):
        return tfp.math.interp_regular_1d_grid(x, x_ref_min=0., x_ref_max=1., 
                                               y_ref=self.K_tf)
        
        
    def por(self, x):
        return tfp.math.interp_regular_1d_grid(x, x_ref_min=0., x_ref_max=1., 
                                               y_ref=self.phi_tf)
        
    
    def lbd(self, s):  # mobilidade (lambda)
        lbd = s**2/self.mu_w + (1-s)**2/self.mu_o
        
        return lbd
    
    
    def ff(self, s):  # fractional flow -> f(s)
        ff = (s**2/self.mu_w)/self.lbd(s)
        
        return ff
    

    def callback(self, loss):
        print('Loss:', loss)


    def train(self, nIter):
        tf_dict = {self.x0_tf: self.x0,
                   self.t0_tf: self.t0,
                   self.x_lb_tf: self.x_lb,
                   self.t_lb_tf: self.t_lb,
                   self.x_ub_tf: self.x_ub,
                   self.t_ub_tf: self.t_lb,
                   self.x_f_tf: self.x_f,
                   self.t_f_tf: self.t_f,
                   self.p0_tf: self.p0,
                   self.s0_tf: self.s0,
                   self.p_lb_tf: self.p_lb,
                   self.p_ub_tf: self.p_ub,
                   self.s_lb_tf: self.s_lb,
                   self.K_tf: self.K,
                   self.phi_tf: self.phi}

        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)

            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                print('It: %d, Loss: %.3e, Time: %.2f' %
                      (it, loss_value, elapsed))
                start_time = time.time()

        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)


    def predict(self, X_star):
        p_star = self.sess.run(self.p0_pred, {self.x0_tf: X_star[:, 0:1], 
                                              self.t0_tf: X_star[:, 1:2]})
        f_p_star = self.sess.run(self.f_p_pred, {self.x_f_tf: X_star[:, 0:1], 
                                                 self.t_f_tf: X_star[:, 1:2],
                                                 self.K_tf: self.K,
                                                 self.phi_tf: self.phi})
        s_star = self.sess.run(self.s0_pred, {self.x0_tf: X_star[:, 0:1], 
                                              self.t0_tf: X_star[:, 1:2]})
        f_s_star = self.sess.run(self.f_s_pred, {self.x_f_tf: X_star[:, 0:1], 
                                                 self.t_f_tf: X_star[:, 1:2],
                                                 self.K_tf: self.K,
                                                 self.phi_tf: self.phi})

        return p_star, f_p_star, s_star, f_s_star
    
    
if __name__ == "__main__":
    N0 = 100
    N_b = 50
    N_f = 10000
    layers =  [2, 100, 100, 100, 100, 2]

    data = scipy.io.loadmat('2phaseflow_spe10_smooth.mat')
    t = data['tt'].flatten()[:, None]
    x = data['x'].flatten()[:, None]
    Exact_p = np.real(data['P_history'])
    Exact_s = np.real(data['S_history'])    
    K = data['K'].flatten()
    phi = data['phi'].flatten()
    mu_w = 0.0855
    mu_o = 0.8550

    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    p_star = Exact_p.T.flatten()[:, None]
    s_star = Exact_s.T.flatten()[:, None]
    
    idx_x = np.random.choice(x.shape[0], N0, replace=False)
    x0 = x[idx_x, :]
    p0 = Exact_p[idx_x, 0:1]
    s0 = 0.1 - 0.1*x0
        
    idx_t = np.random.choice(t.shape[0], N_b, replace=False)
    tb = t[idx_t, :]
    
    p_lb = 0*tb
    p_ub = Exact_p[-1:, idx_t]
    s_lb = Exact_s[0:1, idx_t]
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    lbp = lb + np.array((1/x.size, 0))
    ubp = ub - np.array((1/x.size, 0))
    X_f = lbp + (ubp-lbp)*lhs(2, N_f)

    model = PhysicsInformedNN(x0, p0, s0, p_lb, p_ub, s_lb, tb, 
                              X_f, layers, lb, ub, mu_o, mu_w, K, phi)

    start_time = time.time()
    model.train(1000)
    elapsed = time.time() - start_time
    print('Training time: %.4f' % (elapsed))

    p_pred, f_p_pred, s_pred, f_s_pred = model.predict(X_star)

    error_p = np.linalg.norm(p_star - p_pred, 2)/np.linalg.norm(p_star, 2)
    print('Error p: %e' % (error_p))
    
    error_s = np.linalg.norm(s_star - s_pred, 2)/np.linalg.norm(s_star, 2)
    print('Error s: %e' % (error_s))

    P_pred = griddata(X_star, p_pred.flatten(), (X, T), method='cubic')
    S_pred = griddata(X_star, s_pred.flatten(), (X, T), method='cubic')

    np.savez('2phaseflow_spe10.npz', x0=x0, tb=tb, lb=lb, ub=ub, P_pred=P_pred,
             S_pred=S_pred, Exact_p=Exact_p, Exact_s=Exact_s, t=t, x=x)

In [ ]:
data = np.load('2phaseflow_spe10.npz')
x0 = data['x0']
tb = data['tb']
lb = data['lb']
ub = data['ub']
P_pred = data['P_pred']
S_pred = data['S_pred']
Exact_p = data['Exact_p']
Exact_s = data['Exact_s']
t = data['t']
x = data['x']

X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
X_u_train = np.vstack([X0, X_lb, X_ub])

########################################
####### Pressure #######################
########################################

fig, ax = plt.subplots()
ax.axis('off')

####### Row 0: p(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])

h = ax.imshow(P_pred.T, interpolation='nearest', cmap='viridis',
              extent=[lb[1], ub[1], lb[0], ub[0]],
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:, 1], X_u_train[:, 0], 'kx',
        label = 'Data (%d points)' % (X_u_train.shape[0]),
        markersize = 4, clip_on = False)

line = np.linspace(x.min(), x.max(), 2)[:, None]
ax.plot(t[15]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[60]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[120]*np.ones((2, 1)), line, 'k--', linewidth = 1)

ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
ax.set_title('$p(t,x)$', fontsize = 10)

####### Row 1: p(t,x) slices ##################    
gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.9, wspace=1.0)

ax = plt.subplot(gs1[0, 0])
ax.plot(x, Exact_p[:, 15], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, P_pred[15, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$p(t,x)$')
ax.set_title('$t = %.2f$' % (t[15]), fontsize = 10)
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-1.1, 0.1])

ax = plt.subplot(gs1[0, 1])
ax.plot(x, Exact_p[:, 60], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, P_pred[60, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$p(t,x)$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-1.1, 0.1])
ax.set_title('$t = %.2f$' % (t[60]), fontsize = 10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.8),
          ncol=5, frameon=False)

ax = plt.subplot(gs1[0, 2])
ax.plot(x, Exact_p[:, 120], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, P_pred[120, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$p(t,x)$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-1.1, 0.1])
ax.set_title('$t = %.2f$' % (t[120]), fontsize = 10)

########################################
####### Saturation #####################
########################################

fig, ax = plt.subplots()
ax.axis('off')

####### Row 0: s(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])

h = ax.imshow(S_pred.T, interpolation='nearest', cmap='viridis',
              extent=[lb[1], ub[1], lb[0], ub[0]],
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:, 1], X_u_train[:, 0], 'kx',
        label = 'Data (%d points)' % (X_u_train.shape[0]),
        markersize = 4, clip_on = False)

line = np.linspace(x.min(), x.max(), 2)[:, None]
ax.plot(t[15]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[60]*np.ones((2, 1)), line, 'k--', linewidth = 1)
ax.plot(t[120]*np.ones((2, 1)), line, 'k--', linewidth = 1)

ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
ax.set_title('$S(t,x)$', fontsize = 10)

####### Row 1: s(t,x) slices ##################    
gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.9, wspace=0.5)

ax = plt.subplot(gs1[0, 0])
ax.plot(x, Exact_s[:, 15], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, S_pred[15, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$S(t,x)$')
ax.set_title('$t = %.2f$' % (t[15]), fontsize = 10)
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])

ax = plt.subplot(gs1[0, 1])
ax.plot(x, Exact_s[:, 60], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, S_pred[60, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$S(t,x)$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])
ax.set_title('$t = %.2f$' % (t[60]), fontsize = 10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.8),
          ncol=5, frameon=False)

ax = plt.subplot(gs1[0, 2])
ax.plot(x, Exact_s[:, 120], 'b-', linewidth = 2, label = 'Exact')
ax.plot(x, S_pred[120, :], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$S(t,x)|$')
ax.axis('square')
ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])
ax.set_title('$t = %.2f$' % (t[120]), fontsize = 10) 

## 4. Conclusões
Tomando como base os resultados apresentados acima, pode se concluir que a PINN foi bem sucedida para solucionar o problema proposto, obtendo maior sucesso para o caso simplificado. Tal resultado pode ser encarado como um indicador que este método para solução numérica de equações diferenciais pode ser aplicado com sucesso a sistemas físicos representados por um sistema de equações diferenciais acopladas por um termo quadrático. No entanto, a PINN encontrou dificuldades para lidar com dados reais para a permeabilidade, talvez pela alta variabilidade apresentada por estes dados e por este parâmetro estar dentro de um operador de diferenciação na equação da pressão. 

Os melhores resultados foram obtidos com a PINN com 4 camadas com 100 neurônios cada e com 30000 pontos de colocação. Este resultado era de certa forma esperado, pois mais dados ficaram disponíveis à rede. No entanto, o tempo de treinamento mostrou-se alto ($\sim$ 3 horas) quando comparado ao caso com 10000 pontos de colocação ($\sim$ 30 minutos), cujos erros podem ser considerados baixos em relação aos demais casos testados. 